In [2]:
import os
os.chdir("/mnt/c/Users/Dave/Project/COBS")
import sys
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from cobs import Model, Reward
from cobs import OccupancyGenerator as OG
from tqdm import tqdm
from pprint import pprint

from config import state_names, dist_vars, eplus_naming_dict, eplus_var_types

In [5]:
Model.set_energyplus_folder("/usr/local/EnergyPlus-9-3-0/")
idf_files_path = "/mnt/c/users/Dave/Downloads/idf-sample-files/"

idf_path = idf_files_path+"2020/RefBldgLargeOfficeNew2004_Chicago.idf"
# idf_path = idf_files_path+"2020/RefBldgMediumOfficeNew2004_Chicago.idf"
epw_path = "cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw"

reward = Reward()
ep_model = Model(
    idf_file_name=idf_path,
    weather_file=epw_path,
    eplus_naming_dict=eplus_naming_dict,
    eplus_var_types=eplus_var_types,
    reward=reward,
)

ep_model.set_runperiod(*(14, 1991, 7, 1))
ep_model.set_timestep(4)

In [6]:
# Change default run period
original_run_period = {'Run_Simulation_for_Sizing_Periods': "YES", 
              'Run_Simulation_for_Weather_File_Run_Periods': "NO"}
changed_run_period = {'Run_Simulation_for_Sizing_Periods': "NO", 
              'Run_Simulation_for_Weather_File_Run_Periods': "YES"}
ep_model.edit_configuration("SimulationControl", original_run_period, changed_run_period)

# Add Environment disturbances output variables
print("There should be 28 output varibales before, actually: ", 
      len(ep_model.idf.idfobjects["Output:Variable"]))

# Please note can only run the below code in this cell once
for dist_var in dist_vars:
    ep_model.add_configuration("Output:Variable", {"Variable Name": dist_var, 
                                                   "Reporting Frequency": "Timestep"})
    
print("There are 32 output varibales before, actually: ", len(ep_model.idf.idfobjects["Output:Variable"]))

There should be 28 output varibales before, actually:  28
There are 32 output varibales before, actually:  32


In [4]:
ep_model.reset()
ep_model.step()
ep_model.step()

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.03.21 16:11
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 8
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Wi

{'timestep': 2,
 'time': datetime.datetime(1991, 7, 1, 0, 45),
 'temperature': {'Core_bottom': 24.627573422207853,
  'Core_mid': 26.38595947612868,
  'Core_top': 25.85705630225906,
  'FirstFloor_Plenum': 24.59943039915594,
  'MidFloor_Plenum': 25.28931335344744,
  'Perimeter_bot_ZN_1': 23.341793414231915,
  'Perimeter_bot_ZN_2': 23.29855229047454,
  'Perimeter_bot_ZN_3': 23.329470698694955,
  'Perimeter_bot_ZN_4': 23.32093914906585,
  'Perimeter_mid_ZN_1': 24.75319878037333,
  'Perimeter_mid_ZN_2': 24.69285233400305,
  'Perimeter_mid_ZN_3': 24.735091810125482,
  'Perimeter_mid_ZN_4': 24.718315118884515,
  'Perimeter_top_ZN_1': 24.40277594585291,
  'Perimeter_top_ZN_2': 24.339532562333147,
  'Perimeter_top_ZN_3': 24.374810238447463,
  'Perimeter_top_ZN_4': 24.367768964305945,
  'TopFloor_Plenum': 22.827863783510686},
 'occupancy': {'Core_bottom': 0.0,
  'Core_mid': 0.0,
  'Core_top': 0.0,
  'Perimeter_bot_ZN_1': 0.0,
  'Perimeter_bot_ZN_2': 0.0,
  'Perimeter_bot_ZN_3': 0.0,
  'Perimeter

Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fd98efdc9d0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 526, in _step_callback
    self.wait_for_step.wait()
  File "/usr/lib/python3.8/multiprocessing/synchronize.py", line 349, in wait
    self._cond.wait(timeout)
  File "/usr/lib/python3.8/multiprocessing/synchronize.py", line 261, in wait
    return self._wait_semaphore.acquire(True, timeout)
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fd98efdc9d0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 526, in _step_callback
    self.wait_for_step.wait()
  File "/usr/lib/python3.8/multiprocessing/synchronize.py", line 349, in wait
    self._cond.wait(timeout)
  File "/usr/lib/python3.8/multiprocessing/synchronize.py", l

In [5]:
pd.DataFrame(ep_model.state_history)

""


In [7]:
ep_model.simulate()

['-w', 'cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw', '-d', 'result', 'input.idf']
EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.03.21 16:40
Initializing Response Factors
Calculating CTFs for "MASS NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "UNDERGROUND WALL NON-RES", Construction # 7
Calculating CTFs for "DROPCEILING", Construction # 9
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 10
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Ini

Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <co

Warming up {5}
Warming up {6}
Starting Simulation at 07/01/2013 for RUN PERIOD 1


Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <co

Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <co

...

Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <co

Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <co

Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <cobs.model.Model object at 0x7fc45c6058b0>>
Traceback (most recent call last):
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 550, in _step_callback
    self.prev_reward = self.reward.reward(current_state)
  File "/mnt/c/Users/Dave/project/cobs/cobs/model.py", line 34, in reward
    self.zones = list(state['Lights'].keys())
KeyError: 'Lights'
Exception ignored on calling ctypes callback function: <bound method Model._step_callback of <co

Writing tabular output file results using HTML format.
Writing final SQL reports
EnergyPlus Run Time=00hr 00min  9.33sec


EnergyPlus Completed Successfully.


In [5]:
df = pd.DataFrame(ep_model.state_history)
# Replace year to 1991
df['time'] = df['time'].mask(df['time'].dt.year > 1, df['time'] + pd.offsets.DateOffset(year=1991))
df.index = df['time']
df.drop(columns=['time'], inplace=True)
df.head()

KeyError: 'time'

In [6]:
ep_model.state_history

[]

In [6]:
def get_avg_PPD(df):
    working_time = range(9, 17)
    means = []
    for i in range(len(df)):
        curr = df.iloc[i]
        PPDs = []
        if curr.name.hour in working_time:
            for zone in curr['PPD'].keys():
                if curr['occupancy'][zone] > 0:
                    PPDs.append(curr['PPD'][zone])
        if len(PPDs) == 0:
            mean = 0
        else:
            mean = np.mean(PPDs)
        means.append(mean)
    return np.mean(means)
get_avg_PPD(df)

2.60223730036516

In [7]:
def CalStatics(df):
    meanPPD = get_avg_PPD(df)
    print("Average PPD = ", meanPPD)
    
    HVACPower = np.sum(df["HVAC Power"])
    HVACEnergy = HVACPower*900/1000/3600 # Unit in kWh
    print("Energy Consumed by the HVAC System = {:.2f}kWh".format(HVACEnergy))

CalStatics(df)

Average PPD =  2.60223730036516
Energy Consumed by the HVAC System = 21248.60kWh


In [8]:
results_dir = 'simulation_results'

if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    
df.to_pickle(os.path.join(results_dir, "Sim-chicago.pkl"))
print('Saved!')

Saved!
